In [1]:
import gym
from surface_seg.envs.mcs_env import MCSEnv
import gym.wrappers
import numpy as np
import tensorforce 
import copy

In [2]:
def setup_env(recording=False):
    
    # Set up gym
    MCS_gym = MCSEnv(observation_fingerprints=False, 
                     observation_forces=False,
                    permute_seed=42)
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         "./vid", 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id)%50==0) #every 50, starting at 51
    
    #Convert gym to tensorforce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=400,
                                         visualize=False)
    
    return env

In [3]:
recording_env = setup_env()
recording_env.reset()
action = recording_env.environment.action_space.sample() # Random sampling
recording_env.execute(action) # Prints (dict[state], Termination[Bool], observed reward[float]

found a new local minima! distance=0.14 w energy -0.01


({'fingerprints': array([[ 2.59199195,  3.21267275,  4.88073974,  2.42850498,  3.03348491,
           4.62825488,  1.83311865,  2.36069785,  3.65035984,  0.69936666,
           0.94024857,  1.41191963,  1.0675922 ,  3.84464555,  5.60743085,
           1.17762118,  6.72938299,  3.51393355,  0.30108357,  1.28950832,
           1.90160201,  0.51300125,  2.44087725,  1.37127092],
         [ 4.92024554,  3.57788439,  5.81533489,  4.66182637,  3.34478484,
           5.50221346,  3.68088307,  2.49247141,  4.2886633 ,  1.52196838,
           0.86493521,  1.54719428,  3.32306432,  5.95578054, 11.28954824,
           1.32342954,  7.39410548,  4.60561375,  1.47999726,  2.42762719,
           4.31197389,  0.49715877,  2.44929961,  1.78984176],
         [ 4.9961011 ,  4.87508147,  4.48111756,  4.72706756,  4.60932295,
           4.20854144,  3.69726682,  3.59006324,  3.20052633,  1.41358185,
           1.33728233,  1.18623055,  3.42350745,  9.27811884,  8.17963106,
           3.13804115,  7.9499059

In [4]:
from tensorforce.agents import Agent

agent = Agent.create(
    agent='trpo', 
    environment=setup_env(), 
    batch_size=10, 
    learning_rate=1e-2,
    memory = 40000,
    max_episode_timesteps = 400,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=0.3, decay_steps=1000, decay_rate=0.5
    ))

agent_spec = agent.spec

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent_spec,
    environment=setup_env(),
    max_episode_timesteps=400,
)

%prun runner.run(num_episodes=5)

runner.close()

INFO:tensorflow:Graph was finalized.
